In [21]:
run_no = 0
universe_id = "test"
universe = {
    #"scale": "scale", # "scale", "do-not-scale",
    #"encode_categorical": "one-hot", # "ordinal", "one-hot"
    "model": "logreg", # "logreg", "penalized_logreg", "rf",
    "cutoff": ["quantile_0.15", "quantile_0.30"],
    "exclude_features": "none", # "nationality", "sex", "nationality-sex"
    "exclude_subgroups": "keep-all", # "keep-all", "drop-non-german"
}

output_dir="./output"
seed=0

In [2]:
import json
# Parse universe into dict if it is passed as a string
if isinstance(universe, str):
    universe = json.loads(universe)

In [5]:
# Auto-reload the custom package
%load_ext autoreload
%autoreload 1
%aimport fairness_multiverse

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from fairness_multiverse.universe import UniverseAnalysis

universe_analysis = UniverseAnalysis(
    run_no = run_no,
    universe_id = universe_id,
    universe = universe,
    output_dir=output_dir,
)

In [8]:
import numpy as np
parsed_seed = int(seed)
np.random.seed(parsed_seed)
print(f"Using Seed: {parsed_seed}")

Using Seed: 0


Loading Data

In [11]:
from pathlib import Path
import pandas as pd

# File paths
raw_file = Path("data/raw/siab.csv")
cache_file = Path("data/siab_cached.csv.gz")

# Ensure cache directory exists
cache_file.parent.mkdir(parents=True, exist_ok=True)

# Load with simple caching
if cache_file.exists():
    print(f"Loading SIAB data from cache: {cache_file}")
    siab = pd.read_csv(cache_file, compression='gzip')
else:
    print(f"Cache not found. Reading raw SIAB data: {raw_file}")
    siab = pd.read_csv(raw_file)
    siab.to_csv(cache_file, index=False, compression='gzip')
    print(f"Cached SIAB data to: {cache_file}")

# Now use `siab` DataFrame as needed
print(siab.shape)


Loading SIAB data from cache: data/siab_cached.csv.gz
(643690, 164)


In [12]:
siab

,persnr,year,nrEntry,ltue,employed_before,receipt_leh_before,receipt_lhg_before,se_before,ASU_notue_seeking_before,ASU_other_before,...,minijob_tot_dur_byage,ft_tot_dur_byage,befrist_tot_dur_byage,leih_tot_dur_byage,LHG_tot_dur_byage,LEH_tot_dur_byage,almp_tot_dur_byage,almp_aw_tot_dur_byage,se_tot_dur_byage,seeking1_tot_dur_byage
0,7,2015,1,0,1,0,0,0,1,0,...,0.000000,0.000000,15.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,18,2010,1,1,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,18,2011,2,0,1,0,1,0,1,0,...,2.714286,2.714286,0.000000,0.000000,10.775510,0.000000,8.367347,0.000000,0.000000,9.836735
3,18,2012,3,0,1,0,1,0,1,0,...,4.200000,4.200000,0.000000,0.000000,12.100000,0.000000,9.400000,0.000000,0.000000,9.960000
4,18,2012,4,0,1,0,1,0,1,0,...,5.460000,5.460000,0.000000,0.000000,13.360000,0.000000,10.320000,0.000000,0.000000,10.280000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643685,1827860,2013,1,0,0,0,1,0,0,1,...,0.000000,0.000000,0.000000,0.000000,0.612903,0.000000,0.000000,0.000000,0.000000,0.000000
643686,1827860,2015,2,1,0,0,1,0,1,0,...,0.000000,0.000000,0.212121,0.212121,17.363636,0.000000,8.909091,0.000000,0.000000,7.121212
643687,1827860,2016,3,1,0,0,1,1,0,1,...,0.000000,0.000000,0.294118,0.205882,34.705882,0.000000,10.352941,1.705882,1.705882,23.911765
643688,1827869,2013,1,1,1,0,0,0,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.525424,0.000000,0.000000,0.000000,0.000000


Pre-Processing for Selected Task -> skipped. I think I don't need this

Preprocessing Data

In [23]:
# Exclude protected features
# "exclude_features": "none", # "nationality", "sex", "nationality-sex"

excluded_features = universe["exclude_features"].split("-") # split????
excluded_features_dictionary = {
    "nationality": "maxdeutsch1",
    "nationality_missing": "maxdeutsch.Missing.",
    "sex": "frau1",
}


In [24]:
# Code nice names to column names
excluded_feature_columns = [
    excluded_features_dictionary[f] for f in excluded_features if len(f) > 0 and f != "none"
]

In [25]:
if len(excluded_feature_columns) > 0:
    print(f"Dropping features: {excluded_feature_columns}")
    features.drop(excluded_feature_columns, axis=1, inplace=True)

In [ ]:
# Keep protected features, data from 2010-2014
#X_train_f = siab_train.iloc[:,4:164]

# Drop protected attributes, data from 2010-2014
#X_train_s = X_train_f.drop(
#    columns = ['frau1', 'maxdeutsch1', 'maxdeutsch.Missing.'])